This is the script for retrieving the data needed for day 1 of the HGSFP Gaia lecture

In [1]:
%pylab inline
import gdr2_completeness.tap_queries as tap
from astropy.io import fits
from ezpadova import parsec
import pylab as plt
import math,numpy,copy
from time import sleep

Populating the interactive namespace from numpy and matplotlib


/home/rybizki/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/rybizki/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
## example how to query everything for all sources that have parallax > 20 i.e. are closer than 50pc
Select_what =   """*"""
under_condition = """AND parallax > 20"""
folder = '../data/day_1/gdr2_near/50pc/'
hpx_level = 0
service = 'CDS'

# This actually queries the selected service
tap.tap_query_gdr2_hpx_sliced(service = service, hpx_level = hpx_level, folder = folder,
                              Select_what = Select_what, under_condition = under_condition,
                              verbose = True, test_1st_hpx_only=False)

# Here we combine the files to one structured array
tap.stack_healpix_files(folder)

# And save it in a format readable to topcat
x = np.load(folder + 'result.npy')
fits.writeto(folder + 'nearby_sample_50pc.fits', x)

Healpix level, gaia_factor, #healpixels
0 576460752303423488 12
folder did not exist before, is created now
number of attempt: 1
number of files in folder: 0
number of hpx chunks: 12
0 added to the job list
1 added to the job list
2 added to the job list
3 added to the job list
4 added to the job list
5 added to the job list
6 added to the job list
7 added to the job list
8 added to the job list
9 added to the job list
10 added to the job list
11 added to the job list
total ADQL queries to download: 12


1 / 12 done


10 / 12 done


11 / 12 done


all files have been successfully downloaded to ../data/day_1/


In [3]:
# Same as above but querying the GDR2mock catalogue
Select_what =   """*"""
under_condition = """AND parallax > 10"""
folder = '../data/day_1/gdr2mock_near/'
hpx_level = 0
service = 'GDR2mock'

tap.tap_query_gdr2_hpx_sliced(service = service, hpx_level = hpx_level, folder = folder,
                              Select_what = Select_what, under_condition = under_condition,
                              verbose = True, test_1st_hpx_only=False)

tap.stack_healpix_files(folder)

Healpix level, gaia_factor, #healpixels
0 576460752303423488 12
folder did not exist before, is created now
number of attempt: 1
number of files in folder: 0
number of hpx chunks: 12
0 added to the job list
1 added to the job list
2 added to the job list
3 added to the job list
4 added to the job list
5 added to the job list
6 added to the job list
7 added to the job list
8 added to the job list
9 added to the job list
10 added to the job list
11 added to the job list
total ADQL queries to download: 12


3 / 12 done
8 / 12 done
all files have been successfully downloaded to ../data/day_1/mock/


ValueError: Illegal format `object`.

In [3]:
# Set solar metallicity.
Zsun = 0.0152
# AV grid.
AVgrid = numpy.arange(0., 0.00001, 0.025)
# Set age grid for isochrones.
logt0 =  6.60  # minimal value of log(t/yr)
logt1 = 10.10  # maximal value of log(t/yr)
dlogt =  0.05  # step in log(t/yr) for the sequence
# Loop over extinction.
Output = open('../data/day_1/parsec_isochrones/parsec-auto-grid_bands.csv', 'w')
Output.write('#mass,age,feh,a0,teff,logg,lum,gaia_g,gaia_bp,gaia_rp,ubv_u,ubv_b,ubv_v,ubv_r,ubv_i,ubv_j,ubv_h,ubv_k,sdss_u,sdss_g,sdss_r,sdss_i,sdss_z,2mass_j,2mass_h,2mass_ks,wise_w1,wise_w2,wise_w3,wise_w4\n')

for feh_index,feh in enumerate([-2.0,-1.75,-1.5,-1.25,-1.0,-0.75,-0.5,-0.25,0.0,0.25,0.5]):
    # Write output.

    # From [Fe/H] to Z.
    Z = Zsun*math.pow(10.0, feh)
    # Get magnitudes for zero extinction
    G0   = numpy.nan
    Gbp0 = numpy.nan
    Grp0 = numpy.nan
    for av in AVgrid:
        table = parsec.get_t_isochrones(logt0, logt1, dlogt, Z, ret_table=True, model='parsec12s', phot='gaiaDR2', extinction_av=av)
        ubv = parsec.get_t_isochrones(logt0, logt1, dlogt, Z, ret_table=True, model='parsec12s', phot='ubvrijhk', extinction_av=av)
        ugriz = parsec.get_t_isochrones(logt0, logt1, dlogt, Z, ret_table=True, model='parsec12s', phot='sloan', extinction_av=av)
        wise = parsec.get_t_isochrones(logt0, logt1, dlogt, Z, ret_table=True, model='parsec12s', phot='2mass_spitzer_wise', extinction_av=av)

        #print table.keys()
        
    # Grab photometry.
        assert(numpy.array_equal(table['M_ini'],ubv['M_ini']))
        assert(numpy.array_equal(ugriz['M_ini'],ubv['M_ini']))
        assert(numpy.array_equal(wise['M_ini'],ubv['M_ini']))
        smass = table['M_ini']
        age    = numpy.power(10,table['logageyr'])*1e-9
        teff = table['logTe']
        logg = table['logG']
        lum = table['logLLo']
        G       = table['G']
        Gbp     = table['G_BP']
        Grp     = table['G_RP']
        U = ubv['U'] 
        B = ubv['B']
        V = ubv['V']
        R = ubv['R']
        I = ubv['I']
        J = ubv['J']
        H = ubv['H']
        K = ubv['K']
        u = ugriz['u']
        g = ugriz['g']
        r = ugriz['r']
        i = ugriz['i']
        z = ugriz['z']
        J2 = wise['J']
        H2 = wise['H']
        K2 = wise['Ks']
        W1 = wise['W1']
        W2 = wise['W2']
        W3 = wise['W3']
        W4 = wise['W4']        
        N = len(G)
        # Write outputs.
        for n in range(N):
            Output.write(str(smass[n])+','+str(age[n])+','+str(feh)+','+str(av)+','+str(teff[n])+
                         ','+str(logg[n])+','+str(lum[n])+','+str(G[n])+','+str(Gbp[n])+
                         ','+str(Grp[n])+','+str(U[n])+','+str(B[n])+','+str(V[n])+','+str(R[n])+
             ','+str(I[n])+','+str(J[n])+','+str(H[n])+','+str(K[n])+','+str(u[n])+
             ','+str(g[n])+','+str(r[n])+','+str(i[n])+','+str(z[n])+','+str(J2[n])+
             ','+str(H2[n])+','+str(K2[n])+','+str(W1[n])+','+str(W2[n])+','+str(W3[n])+
             ','+str(W4[n])+'\n')
        Output.flush()
        print("Finished  [Fe/H]="+str(feh)+'   AV='+str(av))
        sleep(2)
    # Close output file.

Output.close()

Interrogating http://stev.oapd.inaf.it...
Interrogating http://stev.oapd.inaf.it...
Interrogating http://stev.oapd.inaf.it...
Interrogating http://stev.oapd.inaf.it...
Finished  [Fe/H]=-2.0   AV=0.0
Interrogating http://stev.oapd.inaf.it...
Interrogating http://stev.oapd.inaf.it...
Interrogating http://stev.oapd.inaf.it...
Interrogating http://stev.oapd.inaf.it...
Finished  [Fe/H]=-1.75   AV=0.0
Interrogating http://stev.oapd.inaf.it...
Interrogating http://stev.oapd.inaf.it...
Interrogating http://stev.oapd.inaf.it...
Interrogating http://stev.oapd.inaf.it...
Finished  [Fe/H]=-1.5   AV=0.0
Interrogating http://stev.oapd.inaf.it...
Interrogating http://stev.oapd.inaf.it...
Interrogating http://stev.oapd.inaf.it...
Interrogating http://stev.oapd.inaf.it...
Finished  [Fe/H]=-1.25   AV=0.0
Interrogating http://stev.oapd.inaf.it...
Interrogating http://stev.oapd.inaf.it...
Interrogating http://stev.oapd.inaf.it...
Interrogating http://stev.oapd.inaf.it...
Finished  [Fe/H]=-1.0   AV=0.0
Int

In [4]:
table.data

rec.array([(0.048067,  6.6 , 0.10906162, 0.1091, -1.5775, 3.3982,  3.5978,  8.714, 10.949, 13.62 ,  9.608, 1.65503716, 0),
           (0.048067,  6.6 , 0.11472474, 0.1147, -1.5454, 3.4008,  3.5982,  8.634, 10.841, 13.481,  9.504, 1.70424283, 0),
           (0.048067,  6.6 , 0.13623628, 0.1362, -1.4165, 3.4112,  3.5856,  8.311, 10.408, 12.921,  9.086, 1.86993372, 0),
           ...,
           (0.048067, 10.05, 1.04517996, 0.9191,  3.3707, 3.444 , -0.2414, -3.657, -0.324,  2.642, -1.682, 3.15261364, 7),
           (0.048067, 10.05, 1.04518032, 0.9183,  3.383 , 3.4419, -0.2625, -3.688, -0.368,  2.601, -1.727, 3.15261364, 7),
           (0.048067, 10.05, 1.04518044, 0.9181,  3.383 , 3.4419, -0.2626, -3.688, -0.368,  2.601, -1.727, 3.15261364, 8)],
          dtype=[('Z', '<f8'), ('logageyr', '<f8'), ('M_ini', '<f8'), ('M_act', '<f8'), ('logLLo', '<f8'), ('logTe', '<f8'), ('logG', '<f8'), ('mbol', '<f8'), ('G', '<f8'), ('G_BP', '<f8'), ('G_RP', '<f8'), ('int_IMF', '<f8'), ('stage', '<i8')])

In [7]:
x = np.genfromtxt('../data/day_1/parsec_isochrones/parsec-auto-grid_bands.csv', 
                  names = True, dtype = None, delimiter=',')

In [9]:
x = x.view(np.recarray)

In [10]:
np.save("../data/day_1/parsec_isochrones/no_ext_grid.npy",x)

In [9]:
x = np.genfromtxt('../data/day_1/DR2_RUWE_V1/table_u0_g_col.txt',names = True,dtype = None, delimiter = ",")
np.save('../data/day_1/DR2_RUWE_V1/table_u0_g_col.npy',x)

In [7]:
x = np.genfromtxt('../data/day_1/DR2_RUWE_V1/table_u0_g.txt',names = True,dtype = None, delimiter = ",")
np.save('../data/day_1/DR2_RUWE_V1/table_u0_g.npy',x)